In [ ]:
import os
os.environ['KAGGLE_USERNAME'] = 'pogusthewhisper'
os.environ['KAGGLE_KEY'] = '755720e5147a6550b4d67a3b66981cb4'

In [ ]:
!kaggle kernels output pogusthewhisper/alb-classify-dataset -p /kaggle/working/
!unzip -q /kaggle/working/dataset.zip
!mv /kaggle/working/kaggle/working/dataset /kaggle/working/
!rm -r /kaggle/working/kaggle && rm /kaggle/working/surgicare-dataset.log

In [ ]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torchvision.models import efficientnet_v2_l, EfficientNet_V2_L_Weights
from torch.utils.data import DataLoader, Subset, WeightedRandomSampler
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.amp import autocast, GradScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
train_dir = '/kaggle/working/dataset/train'
test_dir = '/kaggle/working/dataset/test'

train_tf = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

val_tf = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [ ]:
class WoundClassifier(nn.Module):
    def __init__(self, num_classes=5, dropout=0.4):
        super().__init__()

        base = efficientnet_v2_l(weights=EfficientNet_V2_L_Weights.DEFAULT)
        n_features = base.classifier[1].in_features
        base.classifier = nn.Identity()
        self.backbone = base

        self.shared_head = nn.Sequential(
            nn.Linear(n_features, 512),
            nn.GELU(),
            nn.BatchNorm1d(512),
            nn.Dropout(dropout)
        )

        self.class_head = nn.Sequential(
            nn.Linear(512, 256),
            nn.GELU(),
            nn.BatchNorm1d(256),
            nn.Dropout(dropout),
            nn.Linear(256, num_classes)
        )

        self.layer_groups = [
            self.backbone.features[0:2],
            self.backbone.features[2:4],
            self.backbone.features[4:6],
            self.backbone.features[6:]
        ]

    def forward(self, x):
        x = self.backbone(x)
        x = self.shared_head(x)
        cls_out = self.class_head(x)
        return cls_out

    def freeze_all(self):
        for param in self.backbone.parameters():
            param.requires_grad = False

    def unfreeze_all(self):
        for param in self.backbone.parameters():
            param.requires_grad = True

    def unfreeze_group(self, group_idx):
        for param in self.layer_groups[group_idx].parameters():
            param.requires_grad = True

    def freeze_group(self, group_idx):
        for param in self.layer_groups[group_idx].parameters():
            param.requires_grad = False

In [ ]:
class FocalLossWithSmoothing(nn.Module):
    def __init__(self, gamma=2.0, smoothing=0.1):
        super().__init__()
        self.gamma = gamma
        self.smoothing = smoothing

    def forward(self, logits, targets):
        num_classes = logits.size(1)
        with torch.no_grad():
            true_dist = torch.zeros_like(logits)
            true_dist.fill_(self.smoothing / (num_classes - 1))
            true_dist.scatter_(1, targets.unsqueeze(1), 1.0 - self.smoothing)

        log_probs = F.log_softmax(logits, dim=1)
        probs = torch.exp(log_probs)
        focal = (1 - probs.gather(1, targets.unsqueeze(1)).squeeze(1)) ** self.gamma
        loss = -torch.sum(true_dist * log_probs, dim=1)
        return (focal * loss).mean()

class ClassificationLoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.cls_loss = FocalLossWithSmoothing()

    def forward(self, cls_pred, cls_target):
        loss_cls = self.cls_loss(cls_pred, cls_target)
        return loss_cls, loss_cls.item()

In [ ]:
class EarlyStopping:
    def __init__(self, patience=6, delta=0.0, save_path='best_model.pt'):
        self.patience = patience
        self.delta = delta
        self.best_score = None
        self.counter = 0
        self.early_stop = False
        self.save_path = save_path

    def __call__(self, val_loss, model):
        score = -val_loss
        if self.best_score is None or score > self.best_score + self.delta:
            self.best_score = score
            self.counter = 0
            torch.save(model.state_dict(), self.save_path)
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

In [ ]:
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from torch.utils.data import Subset, DataLoader, WeightedRandomSampler
from collections import Counter
import numpy as np

def create_sampler(subset):
    labels = [subset.dataset.targets[i] for i in subset.indices]
    class_counts = Counter(labels)
    class_weights = {cls: 1.0 / count for cls, count in class_counts.items()}
    weights = [class_weights[label] for label in labels]
    return WeightedRandomSampler(weights, len(weights), replacement=True)

def get_kfold_dataloaders(dataset, k=5, batch_size=32, val_split=0.2):
    y = dataset.targets
    skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)
    folds = []

    for train_val_idx, _ in skf.split(np.arange(len(dataset)), y):
        y_train_val = [y[i] for i in train_val_idx]
        sss = StratifiedShuffleSplit(n_splits=1, test_size=val_split, random_state=42)
        train_idx, val_idx = next(sss.split(train_val_idx, y_train_val))

        train_indices = [train_val_idx[i] for i in train_idx]
        val_indices = [train_val_idx[i] for i in val_idx]

        train_ds = Subset(dataset, train_indices)
        val_ds = Subset(dataset, val_indices)
        train_ds.dataset.transform = train_tf
        val_ds.dataset.transform = val_tf

        train_loader = DataLoader(train_ds, batch_size=batch_size, sampler=create_sampler(train_ds))
        val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False)
        folds.append({'train': train_loader, 'val': val_loader})

    return folds


In [ ]:
def train_top_down(model_class, dataset, num_classes=5, stages=4, epochs_per_stage=10,
                   batch_size=16, base_lr=5e-4, weight_decay=5e-5,
                   patience=5, device='cuda' if torch.cuda.is_available() else 'cpu'):

    folds = get_kfold_dataloaders(dataset, k=5, batch_size=batch_size)
    results = []

    for fold_idx, loaders in enumerate(folds):
        print(f"\nStarting Fold {fold_idx + 1}/5")

        model = model_class(num_classes=num_classes).to(device)
        model.freeze_all()
        model.shared_head.requires_grad_(True)
        model.class_head.requires_grad_(True)

        train_loader = loaders['train']
        val_loader = loaders['val']
        loss_fn = ClassificationLoss()

        for stage in range(stages):
            print(f"\nStage {stage + 1}/{stages}: Unfreezing group {stage}")
            if stage > 0:
                model.unfreeze_group(stage - 1)

            optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()),
                                          lr=base_lr * (0.5 ** stage), weight_decay=weight_decay)
            scheduler = CosineAnnealingLR(optimizer, T_max=epochs_per_stage)
            scaler = GradScaler()
            early_stopper = EarlyStopping(patience=patience,
                                          save_path=f'topdown_model_fold{fold_idx}_stage{stage}.pt')

            for epoch in range(epochs_per_stage):
                model.train()
                train_loss = 0.0

                for inputs, targets in train_loader:
                    inputs = inputs.to(device)
                    cls_targets = targets.to(device).long()

                    optimizer.zero_grad()
                    with autocast(device_type=device):
                        cls_out = model(inputs)
                        loss, _ = loss_fn(cls_out, cls_targets)
                    scaler.scale(loss).backward()
                    scaler.step(optimizer)
                    scaler.update()
                    train_loss += loss.item() * inputs.size(0)

                train_loss /= len(train_loader.dataset)

                model.eval()
                val_loss = 0.0
                all_preds, all_targets = [], []
                with torch.no_grad():
                    for inputs, targets in val_loader:
                        inputs = inputs.to(device)
                        cls_targets = targets.to(device).long()

                        cls_out = model(inputs)
                        loss, _ = loss_fn(cls_out, cls_targets)
                        val_loss += loss.item() * inputs.size(0)
                        preds = torch.argmax(cls_out, dim=1)
                        all_preds.extend(preds.cpu().numpy())
                        all_targets.extend(cls_targets.cpu().numpy())

                val_loss /= len(val_loader.dataset)
                val_acc = accuracy_score(all_targets, all_preds)
                val_f1 = f1_score(all_targets, all_preds, average='macro')
                scheduler.step()

                print(f"[Fold {fold_idx+1} | Stage {stage+1} | Epoch {epoch+1}/{epochs_per_stage}] "
                      f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} "
                      f"| Val Acc: {val_acc:.4f} | Val F1: {val_f1:.4f}")

                early_stopper(val_loss, model)
                if early_stopper.early_stop:
                    print("Early stopping")
                    break

            model.load_state_dict(torch.load(f'topdown_model_fold{fold_idx}_stage{stage}.pt'))

        model.eval()
        all_preds, all_targets = [], []
        with torch.no_grad():
            for inputs, targets in val_loader:
                inputs = inputs.to(device)
                cls_targets = targets.to(device).long()

                cls_out = model(inputs)
                preds = torch.argmax(cls_out, dim=1)
                all_preds.extend(preds.cpu().numpy())
                all_targets.extend(cls_targets.cpu().numpy())

        fold_acc = accuracy_score(all_targets, all_preds)
        fold_f1 = f1_score(all_targets, all_preds, average='macro')
        results.append({'fold': fold_idx + 1, 'accuracy': fold_acc, 'f1': fold_f1})
        print(f"Fold {fold_idx + 1} Accuracy: {fold_acc:.4f}, F1: {fold_f1:.4f}")

    print("\nSummary:")
    for res in results:
        print(f"Fold {res['fold']} - Accuracy: {res['accuracy']:.4f}, F1: {res['f1']:.4f}")
    print(f"\nAvg Accuracy: {np.mean([r['accuracy'] for r in results]):.4f} | "
          f"Avg F1: {np.mean([r['f1'] for r in results]):.4f}")

In [ ]:
%%time
dataset = datasets.ImageFolder(train_dir)
train_top_down(WoundClassifier, dataset, num_classes=len(dataset.classes))

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, f1_score
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import torch

def evaluate_model(model, dataloader, class_names=None, device='cuda'):
    model.eval()
    all_preds = []
    all_targets = []

    with torch.no_grad():
        for inputs, targets in dataloader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            if isinstance(outputs, tuple):
                outputs = outputs[0]
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())

    all_preds = np.array(all_preds)
    all_targets = np.array(all_targets)

    print("\nClassification Report:")
    print(classification_report(
        all_targets, all_preds,
        target_names=class_names,
        zero_division=0
    ))

    cm = confusion_matrix(all_targets, all_preds)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.show()

    per_class_acc = cm.diagonal() / cm.sum(axis=1)
    for idx, acc in enumerate(per_class_acc):
        name = class_names[idx] if class_names else str(idx)
        print(f"Accuracy for class '{name}': {acc:.2%}")

    return {
        'accuracy': np.mean(all_preds == all_targets),
        'f1_macro': f1_score(all_targets, all_preds, average='macro', zero_division=0),
        'per_class_accuracy': per_class_acc
    }

In [ ]:
def evaluate_all_folds(model_class, dataset, stages=4, device='cuda'):
    class_names = dataset.classes
    folds = get_kfold_dataloaders(dataset, k=5, batch_size=32)
    metrics_all = []

    for fold_idx, loaders in enumerate(folds):
        print(f"\n============================")
        print(f"📁 Evaluating Fold {fold_idx + 1}")
        print(f"============================")

        model = model_class(num_classes=len(class_names)).to(device)
        model.load_state_dict(torch.load(f'topdown_model_fold{fold_idx}_stage{stages - 1}.pt'))
        model.eval()

        fold_metrics = evaluate_model(model, loaders['val'], class_names=class_names, device=device)
        metrics_all.append(fold_metrics)

        print(f"Fold {fold_idx + 1} Accuracy: {fold_metrics['accuracy']:.4f}")

    avg_acc = np.mean([m['accuracy'] for m in metrics_all])
    avg_f1 = np.mean([m['f1_macro'] for m in metrics_all])
    print(f"\nAvg Accuracy: {avg_acc:.4f} | Avg F1 Score: {avg_f1:.4f}")

    return metrics_all

In [ ]:
test_dataset = datasets.ImageFolder(test_dir, transform=val_tf)
evaluate_all_folds(WoundClassifier, test_dataset, device='cuda')